In [1]:
import os
import json
from tqdm.auto import tqdm
from urllib.request import urlopen
import html
import re

# Change directory
os.chdir('../../Data/')

In [2]:
df = []
for congress in ["108", "109", "110", "111"]:
    print('Processing {}th congress.'.format(congress))
    offset = -100
    condition = 100
    while condition == 100:
        offset += 100
        response = urlopen("https://api.govinfo.gov/collections/CHRG/2003-01-01T00%3A00%3A00Z?offset=" + str(offset) + "&pageSize=100&congress=" + congress + "&api_key=kQzApyHAZpr4CRWdviBCDTEpCdpqHRWwXlBy6KRk").read().decode('utf-8')
        response = json.loads(response)
        condition = len(response['packages'])
        # Select House and Senate Committe Hearings (packageId starts with CHRG)
        df = df + [x for x in response['packages'] if x not in df and x['docClass'] in ['HHRG', 'SHRG'] and x['packageId'].startswith('CHRG')] 
print('\n{} df have been found.'.format(len(df)))

Processing 108th congress.
Processing 109th congress.
Processing 110th congress.
Processing 111th congress.

12090 hearings have been found.


In [4]:
# Add the text (htm) and mods
for i, hearing in enumerate(tqdm(df)):
    htm_url = "https://api.govinfo.gov/packages/" + hearing['packageId'] + "/htm?api_key=kQzApyHAZpr4CRWdviBCDTEpCdpqHRWwXlBy6KRk"
    mod_url = "https://api.govinfo.gov/packages/" + hearing['packageId'] + "/mods?api_key=kQzApyHAZpr4CRWdviBCDTEpCdpqHRWwXlBy6KRk"
    df[i]['htm'] = urlopen(htm_url).read()
    df[i]['mods'] = urlopen(mod_url).read().decode('utf-8')
print('Text and mods file have been added to the {} df.'.format(len(df)))

100%|██████████| 12090/12090 [4:36:02<00:00,  1.37s/it] 

Text and mods file have been added to the 12090 hearings.


In [3]:
# Create lowercase stripped content
for i, hearing in enumerate(tqdm(df)):
    df[i]['htm_stripped'] =  re.sub(r'[\n\t\s]+', ' ', df[i]['htm'].lower())

In [6]:
# Decode the html mods
for i, hearing in enumerate(tqdm(df)):
    df[i]['mods'] =  html.unescape(df[i]['mods'])

In [7]:
# Replace missing hearing title
df[6094]['title'] = 'Voter Suppression'

In [10]:
# Save the data
with open('CommitteeHearings/hearings_raw.json', 'w') as file:
    json.dump(df, file)